[Reference] https://deap.readthedocs.io/en/master/examples/nsga3.html

In [4]:
import random
import time
import os
import glob
import math
from easydict import EasyDict
import collections
import numpy as np

import networkx as nx

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn    # for hardware tunning (cudnn.benchmark = True)

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from thop import profile
from thop import clever_format

from lr_scheduler import LRScheduler
import logging

## 1. generation pool 구성하기 (Small RWNN 대상)

- stage1, 2, 3에 쓸 random graph 100개 만들기 (가능한 조합수: 100 x 100 x 100) => 1M개의 조합 내에서 GA로 최적의 조합을 빠르게 찾자


- 파일명 형식 = graphmodel_Nodes_K_P
    
    
    e.g. WS_32_4_075

In [5]:
# 실험을 위한 환경 셋팅
run_code = 'test_kyy'
stage_pool_path = './graph_pool' + '/' + run_code + '/'
if not os.path.exists(stage_pool_path):
    os.makedirs(stage_pool_path)
    
# make the log directory
log_path = './logs/' + run_code + '/'
if not os.path.isdir(log_path): os.makedirs(log_path)
    
logging.basicConfig(filename=log_path + 'logging.log', level=logging.INFO)
logging.info('Start to write log.')

In [6]:
##### [모듈화 - 시작] util_graph_new #####

In [7]:
def build_graph(Nodes, args):
    if args.graph_model == 'ER':
        return nx.random_graphs.erdos_renyi_graph(Nodes, args.P)
    elif args.graph_model == 'BA':
        return nx.random_graphs.barabasi_albert_graph(Nodes, args.M)
    elif args.graph_model == 'WS':
        return nx.random_graphs.connected_watts_strogatz_graph(Nodes, args.K, args.P, tries=200)

def save_graph(graph, path):
    nx.write_yaml(graph, path)
    
def load_graph(path):
    return nx.read_yaml(path)

Node = collections.namedtuple('Node', ['id', 'inputs', 'type'])  # typename, field_names

def get_graph_info(graph):
    input_nodes = []
    output_nodes = []
    Nodes = []
    for node in range(graph.number_of_nodes()):
        # node i 에 대해        
        tmp = list(graph.neighbors(node))
        tmp.sort()    # 오름차순 정렬
    
        # node type 정의    
        type = -1    # input node도, output node도 아닌. 그래프의 중간에 매개자처럼 있는 중간 node.
        if node < tmp[0]:
            input_nodes.append(node)
            type = 0    # id 가장 작은 노드보다 작으면, 이건 외부에서 input을 받는 노드. 즉 input node.
        if node > tmp[-1]:
            output_nodes.append(node)
            type = 1    # id 가장 큰 노드보다 크면, 이건 외부로 output 내보내는 노드. 즉 output node.
        
        # dag로 변환 (자신의 id보다 작은 노드들과의 연결만 남기기)
        # [type] 0: input node, 1: output node, -1: input도 output도 아닌, 그래프 중간에 매개자처럼 있는 중간 node
        Nodes.append(Node(node, [n for n in tmp if n < node], type))    # DAG(Directed Acyclic Graph)로 변환
    return Nodes, input_nodes, output_nodes

In [8]:
# 1. random graph 100개 만들기
#     - Nodes 수, K, P의 범위는 임의로 정함.
#     - 일단 WS로만 만듦

num_graph = 100     # 만들고자 하는 graph 수 (pool size)

# 겹치는 그래프가 있을 경우, 100개 이하로 생성될 수 있으므로, saved 된 graph 파일수 체크하며 num_graph개 될때까지 생성.
check_path = glob.glob(stage_pool_path + '*.yaml')
check_file_num = len(check_path)

while check_file_num < num_graph:
    
    Nodes = random.randint(20, 40)  # => [Nodes 값 수정 시 주의] 아래 K값은 Nodes보다 작아야함.
    graph_model = 'WS'
    K = random.randint(4, Nodes-10)  # [min, max] // WS에서는 K nearest. 따라서, 4 ~ 30 random 선택 하도록
    P = round(random.uniform(0.25, 0.75), 2)   # 소수 둘째자리까지만 나오도록 반올림 => 결과 e.g. 0.75

    args = EasyDict({'graph_model': graph_model, 'K':K, 'P':P})

    P_str = str(P)[0] + str(P)[2:]   # 0.75 => 075
    save_file_path = stage_pool_path + graph_model + '_' + str(Nodes) + '_' + str(K) + '_' + P_str + '.yaml'   # e.g. WS_32_4_075
    
    graph = build_graph(Nodes, args)
    save_graph(graph, save_file_path)
    
    check_path = glob.glob(stage_pool_path + '*.yaml')
    check_file_num = len(check_path)

In [9]:
##### [모듈화 - 끝] util_graph_new #####

# => 최종적으로, num_graph와 stage_pool_path 를 인수로 받아서, 해당 path에 num_graph 수 만큼의 그래프 떨궈주는 함수 만들기
#    일단은 정해진 graph_model은 'WS', K, P 는 인수로 받지 말고 구현
#      =>  이후에 확장하기.

## 2. 크로모좀 설계 & fitness function 정의

- 단순 리스트 ( 원소는 0 ~ 저장된 그래프 수, int )


- e.g. [0, 5, 10] => (0번째 그래프, 5번째 그래프, 10번째 그래프)가 순서대로 stage1, 2, 3을 구성하는 그래프

In [10]:
from deap import base, creator
from deap import tools

In [11]:
"""
# fitness function
    input: [0, 5, 10]   하나의 크로모좀.

    1) input인 [0, 5, 10]을 받아서 (0번째, 5번째, 10번째)에 해당하는 그래프 파일 각각 읽어와서 신경망 구축
    
    2) training (임시로 1 epoch. 실제 실험 시, RWNN과 같은 epoch 학습시키기)
    
    3) return flops, val_accuracy

"""

'\n# fitness function\n    input: [0, 5, 10]   하나의 크로모좀.\n\n    1) input인 [0, 5, 10]을 받아서 (0번째, 5번째, 10번째)에 해당하는 그래프 파일 각각 읽어와서 신경망 구축\n    \n    2) training (임시로 1 epoch. 실제 실험 시, RWNN과 같은 epoch 학습시키기)\n    \n    3) return flops, val_accuracy\n\n'

In [12]:
############################
# Classes and tools
############################
# => Min ( -val_accuracy(top_1),  flops )
creator.create('FitnessMin', base.Fitness, weights=(-1.0, -1.0 ))  # name, base (class), attribute // 
creator.create('Individual', list, fitness=creator.FitnessMin)  # creator.FitnessMaxMin attribute로 가짐

In [13]:
############################
# Initialize
############################
IND_SIZE = 3    # 한 individual, 즉 하나의 chromosome은 3개의 graph. 즉, 3개의 stage를 가짐.

toolbox = base.Toolbox()

# toolbox.attribute(0, (num_graph-1)) 이렇게 사용함.
# 즉, 0 ~ (num_grpah - 1) 중 임의의 정수 선택. => 이걸 3번하면 하나의 small graph가 생김
BOUND_LOW = 0
BOUND_UP = num_graph-1
toolbox.register('attr_int', random.randint, BOUND_LOW, BOUND_UP)   # register(alias, method, argument ...)

# toolbox.attribute라는 함수를 n번 시행해서 containter인 creator.individual에 넣은 후 해당 instance를 반환함.
# e.g. [0, 1, 3] 반환
toolbox.register('individual', tools.initRepeat,
                 creator.Individual, toolbox.attr_int, n=IND_SIZE)

toolbox.register('population', tools.initRepeat,
                 list, toolbox.individual)    # n은 생략함. toolbox.population 함수를 뒤에서 실행할 때 넣어줌.

In [14]:
# 함수 테스트
temp_ind = toolbox.individual()
print(temp_ind)

temp_pop_size = 8  # 4의 배수. for using 'tools.selTournamentDCD'
temp_pop = toolbox.population(temp_pop_size)
print(temp_pop)

[14, 73, 37]
[[40, 14, 43], [5, 74, 34], [79, 63, 81], [33, 53, 75], [39, 98, 21], [91, 57, 59], [13, 89, 5], [88, 51, 0]]


In [15]:
##### [모듈화 - 시작] model_new.py #####

In [16]:
class depthwise_separable_conv_3x3(nn.Module):
    def __init__(self, nin, nout, stride):
        # input node 일때, stride = 1; => size 유지
        # input node 아닐 대, stride = 2; =>  (x-1)/2 + 1
        super(depthwise_separable_conv_3x3, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin, kernel_size=3, stride=stride, padding=1, groups=nin)
        self.pointwise = nn.Conv2d(nin, nout, kernel_size=1)  # default: stride=1, padding=0, dilation=1, groups=1, bias=True

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out
    
    
class Triplet_unit(nn.Module):
    def __init__(self, inplanes, outplanes, stride=1):
        super(Triplet_unit, self).__init__()
        self.relu = nn.ReLU()
        self.conv = depthwise_separable_conv_3x3(inplanes, outplanes, stride)
        self.bn = nn.BatchNorm2d(outplanes)

    def forward(self, x):
        out = self.relu(x)
        out = self.conv(out)
        out = self.bn(out)
        return out

    
class Node_OP(nn.Module):
    def __init__(self, Node, inplanes, outplanes):
        super(Node_OP, self).__init__()
        self.is_input_node = Node.type == 0
        self.input_nums = len(Node.inputs)    # 해당 Node에 input으로 연결된 노드의 개수

        # input 개수가 1보다 크면, 여러 input을 합쳐야함.
        if self.input_nums > 1:
            self.mean_weight = nn.Parameter(torch.ones(self.input_nums))  # type: torch.nn.parameter.Parameter
            self.sigmoid = nn.Sigmoid()

        if self.is_input_node:
            self.conv = Triplet_unit(inplanes, outplanes, stride=2)   # Triplet_unit = relu, conv, bn
        else:
            self.conv = Triplet_unit(outplanes, outplanes, stride=1)

    # [참고] nn.Sigmoid()(torch.ones(1)) = 0.7311
    # seoungwonpark source 에서는 torch.zeros()로 들어감. => 0.5
    def forward(self, *input):
        if self.input_nums > 1:
            out = self.sigmoid(self.mean_weight[0]) * input[0]
            for i in range(1, self.input_nums):
                out = out + self.sigmoid(self.mean_weight[i]) * input[i]
        else:
            out = input[0]
        out = self.conv(out)
        return out


class StageBlock(nn.Module):
    def __init__(self, graph, inplanes, outplanes):
        super(StageBlock, self).__init__()
        # graph를 input으로 받아서, Node_OP class. 즉, neural network graph로 전환함.
        self.nodes, self.input_nodes, self.output_nodes = get_graph_info(graph)
        self.nodeop  = nn.ModuleList()    # Holds submodules in a list.
        for node in self.nodes:
            # 각각의 node들을 Node_OP class로 만들어준 뒤, nn.ModuleList()인 self.nodeop에 append 해주기
            self.nodeop.append(Node_OP(node, inplanes, outplanes))

    def forward(self, x):
        results = {}
        # input
        for id in self.input_nodes:
            results[id] = self.nodeop[id](x)  # input x를 먼저 graph's input node에 각각 넣어줌.

        # graph 중간 계산
        for id, node in enumerate(self.nodes):
            # 각각의 노드 id에 대해
            if id not in self.input_nodes:
                # graph's input node가 아니라면, 그래프 내에서 해당 노드의 인풋들인 node.inputs의 output인 results[_id]
                #    => 그 결과를 results[id]에 저장.
                # self.nodeop[id]는 해당 id의 Node_OP. 즉, input들을 받아서 forward(모아서, conv 태우기)하는 것.
                # 따라서, input으로 넣을 때 unpack 함.
                # id 작은 노드부터 result를 차근차근 계산하면서, id를 올라감.
                results[id] = self.nodeop[id](*[results[_id] for _id in node.inputs])

        result = results[self.output_nodes[0]]
        # output
        # graph's output_nodes의 output 들을 평균내기
        for idx, id in enumerate(self.output_nodes):
            if idx > 0:
                result = result + results[id]
        result = result / len(self.output_nodes)
        return result
   
 
# Node_OP -> StageBlock class 정의해놓고,
# conv2, conv3, conv4에 각각 random graph 생성해서 모듈로 추가함
# e.g.
#  graphs = EasyDict({'stage_1': stage_1_graph,
#                     'stage_2': stage_2_graph,
#                     'stage_3': stage_3_graph
#  })   # stage_1_graph = 해당 graph 파일의 path
# channels = 109

class RWNN(nn.Module):
    def __init__(self, net_type, graphs, channels, num_classes=1000):
        super(RWNN, self).__init__()
        # 논문에서도 conv1 쪽은 예외적으로 Conv-BN 이라고 언급함. (나머지에서는 Conv-ReLU-BN 을 conv 로 표기) 
        self.conv1 = depthwise_separable_conv_3x3(3, channels // 2, 2)    # nin, nout, stride
        self.bn1 = nn.BatchNorm2d(channels // 2)
    
        # 채널수 변화도, 논문에서처럼 conv2: C, conv3: 2C, conv4: 4C, conv5: 8C    
        if net_type == 'small':
            self.conv2 = Triplet_unit(channels // 2, channels, 2)    # inplanes, outplanes, stride=2

            self.conv3 = StageBlock(graphs.stage_1, channels, channels)
 
            self.conv4 = StageBlock(graphs.stage_2, channels, channels *2)   

            self.conv5 = StageBlock(graphs.stage_3, channels * 2, channels * 4)

            self.relu = nn.ReLU()
            self.conv = nn.Conv2d(channels * 4, 1280, kernel_size=1)   # 마지막에 1x1 conv, 1280-d
            self.bn2 = nn.BatchNorm2d(1280)
        
        #######################################
        # 원 코드에서 regular 부분 지움
        #######################################

        self.avgpool = nn.AvgPool2d(7, stride=1)  # 마지막은 global average pooling
        self.fc = nn.Linear(1280, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))        
        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)

        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.relu(x)
        x = self.conv(x)
        x = self.bn2(x)
        x = self.relu(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [17]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    # torch.topk : input, k, dim=None, largest=True, sorted=True => returns top k element
    # returns values list & indices list
    _, pred = output.topk(maxk, 1, True, True)    
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))   # torch.eq: Computes element-wise equality

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)   # input, dim,
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [18]:
# Train for one epoch
def train(train_loader, model, criterion, optimizer, lr_scheduler, epoch, print_freq):
    batch_time = AverageMeter()
    epoch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        lr_scheduler.update(i, epoch)
        
        target = target.cuda(async=True)
        
        input_var = torch.autograd.Variable(input.cuda())
        target_var = torch.autograd.Variable(target)
        
        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output, target, topk=(1, 5))

        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))
        top5.update(prec5.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('\t - Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                epoch, i, len(train_loader), batch_time=batch_time,
                loss=losses, top1=top1, top5=top5))
            
#             logging.info('Epoch: [{0}][{1}/{2}]\t'
#                   'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
#                   'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
#                   'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
#                   'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
#                 epoch, i, len(train_loader), batch_time=batch_time,
#                 data_time=data_time, loss=losses, top1=top1, top5=top5))
            
            niter = epoch * len(train_loader) + i
#             writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], niter)
#             writer.add_scalar('Train/Avg_Loss', losses.avg, niter)
#             writer.add_scalar('Train/Avg_Top1', top1.avg / 100.0, niter)
#             writer.add_scalar('Train/Avg_Top5', top5.avg / 100.0, niter)

In [19]:
def validate(val_loader, model, criterion, epoch):
#     batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        start = time.time()
        for i, (input, target) in enumerate(val_loader):
            target = target.cuda(async=True)
            input_var = torch.autograd.Variable(input.cuda())
            target_var = torch.autograd.Variable(target)

            # compute output
            output = model(input_var)
            loss = criterion(output, target_var)

            # measure accuracy and record loss
            prec1, prec5 = accuracy(output, target, topk=(1, 5))

            losses.update(loss.item(), input.size(0))
            top1.update(prec1.item(), input.size(0))
            top5.update(prec5.item(), input.size(0))

            # measure elapsed time
#             batch_time.update(time.time() - end)   # 처음에 end랑 batch_time도 바꿈
#             end = time.time()

#             if i % args.print_freq == 0:
#                 print('Test: [{0}/{1}]\t'
#                       'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
#                       'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
#                       'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
#                       'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
#                     i, len(val_loader), batch_time=batch_time, loss=losses,
#                     top1=top1, top5=top5))
                
#                 logging.info('Test: [{0}/{1}]\t'
#                       'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
#                       'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
#                       'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
#                       'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
#                     i, len(val_loader), batch_time=batch_time, loss=losses,
#                     top1=top1, top5=top5))          
                
        # measure elapsed time
        validation_time = time.time() - start

        print('Validation_time {validation_time:.3f} Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
              .format(validation_time=validation_time, top1=top1, top5=top5))

        niter = (epoch + 1)
#         writer.add_scalar('Eval/Avg_Loss', losses.avg, niter)
#         writer.add_scalar('Eval/Avg_Top1', top1.avg / 100.0, niter)
#         writer.add_scalar('Eval/Avg_Top5', top5.avg / 100.0, niter)

    return top1.avg

In [20]:
############################
# Operators
############################
def evaluate(individual, args_train):  # individual
    # list 형식의 individual 객체를 input으로 받음   e.g. [0, 4, 17] 
    # 1) load graph
    total_graph_path = glob.glob(stage_pool_path + '*.yaml')    # list
    
    stage_1_graph = load_graph(total_graph_path[individual[0]])
    stage_2_graph = load_graph(total_graph_path[individual[1]])
    stage_3_graph = load_graph(total_graph_path[individual[2]])
    
    graphs = EasyDict({'stage_1': stage_1_graph,
                       'stage_2': stage_2_graph,
                       'stage_3': stage_3_graph
                      })

    # 2) build RWNN
    channels = 109
    NN_model = RWNN(net_type='small', graphs=graphs, channels=channels)
    NN_model.cuda()
    
    ###########################
    # Flops 계산 - [Debug] nn.DataParallele (for multi-gpu) 적용 전에 확인.
    ###########################
    input_flops = torch.randn(1, 3, 224, 224).cuda()
    flops, params = profile(NN_model, inputs=(input_flops, ), verbose=False)
    
    # 3) Prepare for train
    NN_model = nn.DataParallel(NN_model)  # for multi-GPU
    
    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda()

    optimizer = torch.optim.SGD(NN_model.parameters(), args_train.base_lr,
                                momentum=args_train.momentum,
                                weight_decay=args_train.weight_decay)
    
    start_epoch  = 0
    best_prec1 = 0    
    
    cudnn.benchmark = True    # This flag allows you to enable the inbuilt cudnn auto-tuner to find the best algorithm to use for your hardware.  
    
    ###########################
    # Dataset & Dataloader
    ###########################
    train_transform = transforms.Compose(
        [
            transforms.RandomHorizontalFlip(),   # 추가함
            transforms.Resize(224),    # 추가함.  imagenet dataset과 size 맞추기
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # rescale 0 ~ 1 => -1 ~ 1
        ])

    val_transform = transforms.Compose(
        [
            transforms.Resize(224),    # 추가함.  imagenet dataset과 size 맞추기
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # rescale 0 ~ 1 => -1 ~ 1
        ])


    # 이미 다운 받아놨으니 download=False
    # 데이터가 없을 경우, 처음엔느 download=True 로 설정해놓고 실행해주어야함
    train_dataset = torchvision.datasets.CIFAR10(root='D:\data\cifar10', train=True,
                                            download=True, transform=train_transform)

    val_dataset = torchvision.datasets.CIFAR10(root='D:\data\cifar10', train=False,
                                           download=True, transform=val_transform)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args_train.batch_size,
                                              shuffle=True, num_workers=args_train.workers)  

    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=args_train.batch_size,
                                             shuffle=False, num_workers=args_train.workers)    
    
    ###########################
    # Train
    ###########################
    niters = len(train_loader)

    lr_scheduler = LRScheduler(optimizer, niters, args_train)  # (default) args.step = [30, 60, 90], args.decay_factor = 0.1, args.power = 2.0    
    
    for epoch in range(start_epoch, args_train.epochs):
        # train for one epoch
        train(train_loader, NN_model, criterion, optimizer, lr_scheduler, epoch, args_train.print_freq)

        # evaluate on validation set
        prec1 = validate(val_loader, NN_model, criterion, epoch)
        
        # remember best prec@1 and save checkpoint
#         is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
    
    return -best_prec1, flops   # Min (-val_accuracy, flops) 이므로 val_accuracy(top1)에 - 붙여서 return

In [21]:
##### [모듈화 - 끝] model_new.py #####

In [22]:
##### [모듈화 - 시작] GA_operator_new.py #####

In [23]:
# 기존 mutUniformInt에 xrange() 함수가 사용됐어서, range로 수정함.
import random

from itertools import repeat
from collections import Sequence

def mutUniformInt_custom(individual, low, up, indpb):
    """Mutate an individual by replacing attributes, with probability *indpb*,
    by a integer uniformly drawn between *low* and *up* inclusively.
    :param individual: :term:`Sequence <sequence>` individual to be mutated.
    :param low: The lower bound or a :term:`python:sequence` of
                of lower bounds of the range from wich to draw the new
                integer.
    :param up: The upper bound or a :term:`python:sequence` of
               of upper bounds of the range from wich to draw the new
               integer.
    :param indpb: Independent probability for each attribute to be mutated.
    :returns: A tuple of one individual.
    """
    size = len(individual)
    if not isinstance(low, Sequence):
        low = repeat(low, size)
    elif len(low) < size:
        raise IndexError("low must be at least the size of individual: %d < %d" % (len(low), size))
    if not isinstance(up, Sequence):
        up = repeat(up, size)
    elif len(up) < size:
        raise IndexError("up must be at least the size of individual: %d < %d" % (len(up), size))

    for i, xl, xu in zip(range(size), low, up):
        if random.random() < indpb:
            individual[i] = random.randint(xl, xu)

    return individual,

In [24]:
##### [모듈화 - 끝] GA_operator_new.py #####

In [25]:
# define 'args_train' for evaluation
args_train = EasyDict({
    'lr_mode': 'cosine',
    'warmup_mode': 'linear',    # default
    'base_lr': 0.1,
    'momentum': 0.9, 
    'weight_decay': 0.00005 ,
    'print_freq': 100,

    'epochs': 2,
    'batch_size': 128,

    'workers': 2,

    'warmup_epochs': 0,
    'warmup_lr': 0.0,
    'targetlr': 0.0

})

# toolbox.mate(), toolbox.select() 등의 이름으로 해당 함수에 접근할 수 있음.
toolbox.register('mate', tools.cxTwoPoint)  # crossover

toolbox.register('mutate', mutUniformInt_custom, low=BOUND_LOW, up=BOUND_UP)
# indpb: 뒤쪽에서 쓸 때, MUTPB로 넣어줌. individual의 각 원소에 mutation 적용될 확률
# indpb – Independent probability for each attribute to be mutated.

toolbox.register('select', tools.selNSGA2, nd='standard')  # selection.  // k – The number of individuals to select. k는 함수 쓸 때 받아야함
# => return A list of selected individuals.
# tools.selTournament의 arguement 중 torunsize는 미리 지정해줌

toolbox.register('evaluate', evaluate, args_train=args_train)

In [26]:
# 함수 test 
ind1 = temp_pop[0]
ind2 = temp_pop[1]
print("ind1:", ind1)
print("ind2:", ind2)

cx_results = toolbox.mate(ind1, ind2)  # crossover
print("cx_results:", cx_results)

MUTPB = 0.8  # test를 위해 높게 잡음
mu_results = toolbox.mutate(ind1, indpb=MUTPB)
print("mu_results (ind1):", mu_results)

val_accuracy, flops = toolbox.evaluate(ind1)

print()
print(val_accuracy, flops)

ind1: [40, 14, 43]
ind2: [5, 74, 34]
cx_results: ([40, 14, 34], [5, 74, 43])
mu_results (ind1): ([40, 14, 63],)
Files already downloaded and verified
Files already downloaded and verified


RuntimeError: CUDA out of memory. Tried to allocate 41.75 MiB (GPU 0; 8.00 GiB total capacity; 5.95 GiB already allocated; 9.94 MiB free; 8.61 MiB cached)

In [ ]:
# 함수 test 
# This is just to assign the crowding distance to the individuals's fitness
# no actual selection is done
temp_pop = toolbox.select(temp_pop, len(temp_pop))
print("temp_pop:\n  ", temp_pop)

# Vary the population
temp_offspring = tools.selTournamentDCD(temp_pop, len(temp_pop))  # selTournamentDCD: individuals length must be a multiple of 4
temp_offspring = [toolbox.clone(ind) for ind in temp_offspring]
print("temp_offspring:\n  ", temp_offspring)

In [ ]:
"""
4. Algorithms
 For the purpose of completeness we will develop the complete generational algorithm.
"""

POP_SIZE = 8    # population size
NGEN = 10    # number of Generation
CXPB = 0.5    # crossover probability 
MUTPB = 0.5    # mutation probability


# log에 기록할 stats
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

logbook = tools.Logbook()
logbook.header = "gen", "evals", "min", "max", "evals_time", "gen_time"

# population 생성.  (toolbox.population은 creator.Individual n개를 담은 list를 반환. (=> population)
print("Initialion starts ...")
pop = toolbox.population(n=POP_SIZE)

# Evaluate the individuals with an invalid fitness
invalid_ind = [ind for ind in pop if not ind.fitness.valid]
fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)    # .evaluate는 tuple을 반환. 따라서 fitnesses는 튜플을 원소로 가지는 list
for ind, fit in zip(invalid_ind, fitnesses):
    ind.fitness.values = fit   # ind.fitness.values = (val_accuracy, flops) 튜플

# This is just to assign the crowding distance to the individuals
# no actual selection is done
pop = toolbox.select(pop, len(pop))

record = stats.compile(pop)
logbook.record(gen=0, evals=len(invalid_ind), **record)
print(logbook.stream)
print("Initialization is finished.\n")

# Begin the generational process
for gen in range(1, NGEN):
    print("#####", gen, "th generation starts")
    start_gen = time.time()
    # Vary the population
    offspring = tools.selTournamentDCD(pop, len(pop))
    offspring = [toolbox.clone(ind) for ind in offspring]

    for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        if random.random() <= CXPB:
            toolbox.mate(ind1, ind2)

        toolbox.mutate(ind1, indpb=MUTPB)
        toolbox.mutate(ind2, indpb=MUTPB)
        del ind1.fitness.values, ind2.fitness.values

    # Evaluate the individuals with an invalid fitness
    print("##### Evaluation starts")
    start_time = time.time()
    
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
        
    eval_time_for_one_generation = time.time() - start_time        
    print("##### Evaluation ends (Time : %.3f)" % eval_time_for_one_generation)
    
    # Select the next generation population
    pop = toolbox.select(pop + offspring, POP_SIZE)
    
    gen_time = time.time() - start_gen
    print('##### [gen_time: %.3fs]' % gen_time, gen, 'th generation is finished.')
    
    record = stats.compile(pop)
    logbook.record(gen=gen, evals=len(invalid_ind), **record,
                   evals_time=eval_time_for_one_generation, gen_time=gen_time)
    
    logging.info('Gen [%03d/%03d] -- evals: %03d, evals_time: %.4fs, gen_time: %.4fs' % (gen, NGEN, len(invalid_ind), eval_time_for_one_generation, gen_time))    
    print(logbook.stream)
    


In [ ]:
logbook

In [ ]:
#########################
# logbook save 하기 - 매 generation 지날때마다 해당 파일에 덮어쓰도록.
# save paths는 run_code 붙여서 logs 폴더에
#########################

In [ ]:
"""
- 190813
- POP_SIZE = 8, EPOCH = 2

##### [gen_time: 3253.266s] 1 th generation is finished.
1  	8    	[-6.49600000e+01  1.46135194e+09]	[-5.61200000e+01  1.74673242e+09]	3253.27   	3253.27 

##### [gen_time: 3276.617s] 2 th generation is finished.
2  	8    	[-6.49600000e+01  1.38388634e+09]	[-5.61200000e+01  1.74673242e+09]	3276.62   	3276.62 

##### [gen_time: 3054.050s] 3 th generation is finished.
3  	8    	[-6.60300000e+01  1.28832512e+09]	[-5.83200000e+01  1.69092864e+09]	3054.05   	3054.05 

- EPOCH = 10으로 늘리면, 대략 5배 -> POP_SIZE = 8 일때, 1 gen 당 15,000초 (=250분. 4시간 10분)

=> 3th generation에서 끝남. why?

+ log 쓰는 방법. log book 말고, 다른 파일에 write해서 save 될 수 있도록.

"""

## RWNN 코드 연습 (보관용)  -----------------------------------------------------------

In [ ]:
def build_graph_WS(Nodes, K, P):
    return nx.random_graphs.connected_watts_strogatz_graph(Nodes, K, P, tries=200)

In [ ]:
Node = collections.namedtuple('Node', ['id', 'inputs', 'type'])  # typename, field_names

def get_graph_info(graph):
    input_nodes = []
    output_nodes = []
    Nodes = []
    for node in range(graph.number_of_nodes()):
        # node i 에 대해        
        tmp = list(graph.neighbors(node))
        tmp.sort()    # 오름차순 정렬
    
        # node type 정의    
        type = -1    # input node도, output node도 아닌. 그래프의 중간에 매개자처럼 있는 중간 node.
        if node < tmp[0]:
            input_nodes.append(node)
            type = 0    # id 가장 작은 노드보다 작으면, 이건 외부에서 input을 받는 노드. 즉 input node.
        if node > tmp[-1]:
            output_nodes.append(node)
            type = 1    # id 가장 큰 노드보다 크면, 이건 외부로 output 내보내는 노드. 즉 output node.
        
        # dag로 변환 (자신의 id보다 작은 노드들과의 연결만 남기기)
        # [type] 0: input node, 1: output node, -1: input도 output도 아닌, 그래프 중간에 매개자처럼 있는 중간 node
        Nodes.append(Node(node, [n for n in tmp if n < node], type))    # DAG(Directed Acyclic Graph)로 변환
    return Nodes, input_nodes, output_nodes

In [ ]:
Nodes = 32
K = 4
P = 0.75

sample_graph = build_graph_WS(Nodes, K, P)

In [ ]:
list(sample_graph.neighbors(0))   # 0번 노드의 neightbor

In [ ]:
Nodes, input_nodes, output_nodes = get_graph_info(sample_graph)

In [ ]:
Nodes   # list

In [ ]:
print(len(Nodes))
print(input_nodes)
print(output_nodes)

In [ ]:
# save Nodes
import pickle

with open('Nodes_1', 'wb') as fp:
    pickle.dump(Nodes, fp)

In [ ]:
with open ('Nodes_1', 'rb') as fp:
    temp = pickle.load(fp)

In [ ]:
temp